In [2]:
import pandas as pd

In [ ]:
def describe_df(df : pd.DataFrame):
    """
    Genera una tabla resumen con una descripción de las variables del DataFrame.

    Argumentos
    ----------
    `df` : pandas.DataFrame
        El DataFrame de entrada cuyas columnas vamos a analizar.

    Retorna
    -------
    pandas.DataFrame
        Una tabla resumen con los siguientes datos por fila:
        - DATA_TYPE: El tipo de dato (en python) de cada variable.
        - MISSINGS (%): El porcentaje de nulos/missings.
        - UNIQUE_VALUES: El número de valores únicos de cada columna.
        - CARDIN (%): La cardinalidad de cada variable.

    Notas
    -----
    El DataFrame de salida tiene por columnas las del DataFrame de entrada,
    y las métricas que acabamos de describir como filas.
    """

    # Fila 1: tipo de datos
    data_type = df.dtypes.astype(str)

    # Fila 2: porcentaje de missings
    missings = df.isnull().mean() * 100

    # Fila 3: número de valores únicos
    unique_values = df.nunique()

    # Fila 4: cardinalidad (en %)
    cardin = round((unique_values / len(df)) * 100, 2)

    # Creación del dataframe
    resumen = pd.DataFrame(
        [data_type, missings, unique_values, cardin],
        index=["DATA_TYPE", "MISSINGS (%)", "UNIQUE_VALUES", "CARDIN (%)"]
    )

    return resumen

In [20]:
df_titanic = pd.read_csv("./data/titanic.csv")
prueba_descripcion = describe_df(df_titanic)
display(prueba_descripcion)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
DATA_TYPE,int64,int64,object,float64,int64,int64,float64,object,object,object,bool,object,object,object,bool
MISSINGS (%),0.0,0.0,0.0,19.86532,0.0,0.0,0.0,0.224467,0.0,0.0,0.0,77.216611,0.224467,0.0,0.0
UNIQUE_VALUES,2,3,2,88,7,7,248,3,3,3,2,7,3,2,2
CARDIN (%),0.22,0.34,0.22,9.88,0.79,0.79,27.83,0.34,0.34,0.34,0.22,0.79,0.34,0.22,0.22


In [ ]:
def tipifica_variables(df : pd.DataFrame, umbral_categoria : int = 10, umbral_continua : float = 20):
    """
    Sugiere una clasificación para el tipo de cada variable basada en la cardinalidad y en el número de valores únicos.

    Argumentos
    ----------
    `df` : pandas.DataFrame
        El DataFrame de entrada cuyas variables vamos a clasificar.
    `umbral_categoria` : int, default=10
        Máximo número de valores únicos para considerar una variable como categórica.
    `umbral_continua` : float, default=20
        Porcentaje de cardinalidad a partir del cual consideramos una variable como numérica continua.

    Retorna
    -------
    pandas.DataFrame

    Un DataFrame con dos columnas:
    - `nombre_variable`: el nombre de cada variable del DataFrame original.
    - `tipo_sugerido`: el tipo de variable en el que se clasifica:
        * Binaria.
        * Categorica.
        * Numerica Discreta.
        * Numerica Continua.

    Notas
    -----
    - Si una variable tiene exactamente dos valores únicos, se clasifica como "Binaria".
    - Si el número de valores únicos es menor que `umbral_categoria`, se clasifica como "Categorica".
    - Si el numero de valores unicos es mayor o igual que `umbral_categoria`:
        * Si la cardinalidad es mayor o igual que `umbral_continua`, se clasifica como "Numerica Continua".
        * Si no (cardinalidad menor que `umbral_continua`), se clasifica como "Numerica Discreta".
    """

    tabla_tipificacion = []

    for col in df.columns: # Recorremos las columnas del df de entrada

        # Calculamos el número de valores únicos y la cardinalidad de cada variable
        valores_unicos = df[col].nunique()
        cardinalidad = (valores_unicos / len(df)) * 100

        # Clasificamos según los criterios del enunciado, usando los umbrales de entrada
        if valores_unicos == 2:
            tipo = "Binaria"
        elif valores_unicos < umbral_categoria:
            tipo = "Categorica"
        elif valores_unicos >= umbral_categoria:
            if cardinalidad >= umbral_continua:
                tipo = "Numerica Continua"
            else:
                tipo = "Numerica Discreta"

        # Añadimos la fila con la variable y su clasificación a lo que sera el DataFrame de salida
        tabla_tipificacion.append({"nombre_variable": col, "tipo_sugerido": tipo})

    return pd.DataFrame(tabla_tipificacion)

In [22]:
prueba_tipificacion = tipifica_variables(df_titanic)
display(prueba_tipificacion)

,nombre_variable,tipo_sugerido
0,survived,Binaria
1,pclass,Categorica
2,sex,Binaria
3,age,Numerica Discreta
4,sibsp,Categorica
5,parch,Categorica
6,fare,Numerica Continua
7,embarked,Categorica
8,class,Categorica
9,who,Categorica
